# Library

In [1]:
import numpy as np
import csv
from sentence_transformers import SentenceTransformer
from math import sqrt, pow
from gensim.parsing.preprocessing import preprocess_string, remove_stopwords, stem_text
from gensim.utils import simple_preprocess, tokenize
from gensim.models import Word2Vec, FastText

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import to_categorical


from sklearn.decomposition import PCA
from matplotlib import pyplot as plt


2022-12-22 19:57:16.950167: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-22 19:57:19.039021: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2022-12-22 19:57:19.039372: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2022-12-22 19:57:19.039406: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nv

# Code

## Read data

In [2]:
test_data_AD = './Data/test_data.csv'
train_data_AD = './Data/train_data.csv'
valid_data_AD = './Data/valid_data.csv'

In [3]:
#Test Data

file = open(test_data_AD)

csvreader = csv.reader(file)

header = []
header = next(csvreader)
header

test_data = []
for row in csvreader:
        test_data.append(row)
test_data

file.close()

In [4]:
#Train Data

file = open(train_data_AD)

csvreader = csv.reader(file)

header = []
header = next(csvreader)
header

train_data = []
for row in csvreader:
        train_data.append(row)
train_data

file.close()

In [5]:
#Validation Data

file = open(valid_data_AD)

csvreader = csv.reader(file)

header = []
header = next(csvreader)
header

valid_data = []
for row in csvreader:
        valid_data.append(row)
valid_data

file.close()

## Unique Train data

In [6]:
qid2 = []
for i in range(len(train_data)) :
    qid2.append(train_data[i][2])

unique_res = list(np.unique(qid2))

train_sentence = []
for id in unique_res :
    index = qid2.index(id)
    train_sentence.append(train_data[index][4].lower())

print(len(qid2))
print(len(train_sentence))

37250
16663


In [7]:
indexes = [train_sentence.index(x) for x in set(train_sentence)]
train_sentence = list(set(train_sentence))

print(len(indexes))
print(len(train_sentence))

16641
16641


In [8]:
unique_qid2 = []
for i in indexes :
    unique_qid2.append(unique_res[i])

len(unique_qid2)

16641

## Unique Test questions

In [9]:
qid1 = []
for i in range(len(test_data)) :
    qid1.append(test_data[i][1])

unique_qid1 = list(np.unique(qid1))

test_sentence = []
for id in unique_qid1 :
    index = qid1.index(id)
    test_sentence.append(test_data[index][3].lower())

print(len(qid1))
print(len(test_sentence))

980
146


## Unique Valid questions

In [10]:
qid3 = []
for i in range(len(valid_data)) :
    qid3.append(valid_data[i][1])

unique_qid3 = list(np.unique(qid3))

valid_sentence = []
for id in unique_qid3 :
    index = qid3.index(id)
    valid_sentence.append(valid_data[index][4].lower())

print(len(qid3))
print(len(valid_sentence))

1010
173


## Assessment

#### MAP

In [11]:
def MAP(precision) :
    if len(precision) == 0 :
        return 0
    
    s = 0
    for pre in precision :
        if len(pre) > 0 :
            x = 0
            for i in pre :
                x += i
            s += ( x / len(pre) )
        
    return s / len(precision)

#### P@k

In [12]:
def Precision5(precision) :
    if len(precision) < 5 :
        return 0
    return precision[4]

In [13]:
def Precision10(precision) :
    if len(precision) < 10 :
        return 0
    return precision[9]

#### MRR

In [14]:
def MRR(reciprocal_rank, size) :
    if len(reciprocal_rank) == 0 :
        return 0
    
    s = 0
    for rr in reciprocal_rank :
        s += rr

    return s / size

#### Calculate

In [15]:
def assessment(vector) :
    precision = []
    reciprocal_rank = []
    for i in range(len(vector)) :
        sort_index = np.argsort(vector[i])[::-1]
        sort_index = sort_index[0:10]

        index_pos_list = [ j for j in range(len(qid1)) if qid1[j] == unique_qid1[i] ]
        test_s = [test_data[j][-2].lower() for j in index_pos_list]
    
        pr = []
        sum = 0
        h = 0
        flag = True
        for index in sort_index :
            for z in range(len(train_data)) :
              if qid2[z] == unique_qid2[index] :
                train_s = train_data[z][-2].lower()
                if train_s in test_s :
                    sum += 1
                    pr.append(sum/(h + 1))
                    if flag :
                        flag = False
                        reciprocal_rank.append(1/(h + 1))
                h += 1
        precision.append(pr)
    
    
    for i in range(len(precision)) :
        print('Precision@5 for query ' + str(i) + ' = ' + str(Precision5(precision[i])) )
        print('Precision@10 for query ' + str(i) + ' = ' + str(Precision10(precision[i])) )
    
    print('MAP = ' + str(MAP(precision)) )
    print('MRR = ' + str(MRR(reciprocal_rank, len(precision))) )

## Preprocess

In [16]:
def remove_stop_words(document) :
    tokens = [list(tokenize(doc, lower=True)) for doc in document]
    tokens = [preprocess_string(" ".join(doc), [remove_stopwords, stem_text]) for doc in tokens]
    return tokens

In [17]:
def remove_stop_words2(sentences) :
    for i in range(len(sentences)) :
        sentences[i] = remove_stopwords(sentences[i].lower())
    
    return sentences

## Word2Vector

In [22]:
sentences = remove_stop_words(train_sentence)
sentences.extend(remove_stop_words(valid_sentence))

# train word2vec model
model_w2v = Word2Vec(sentences, min_count=1, vector_size=100, window=5, epochs=10)
# model.save("word2vec.model")
# model = Word2Vec.load("word2vec.model")

# access vector for one word
print(model_w2v.wv['zombi'])

# get other similar words
# sims = model.wv.most_similar('zombi', topn=10)


#list the vocabulary words
words_w2v = list(model_w2v.wv.index_to_key)

print(len(words_w2v))

[-0.07914685  0.13173008  0.03685642  0.03331754  0.05163226 -0.17804459
  0.08465194  0.21255319 -0.07343437 -0.06076741 -0.07861451 -0.1378414
 -0.01576091  0.04419099  0.05559474 -0.09130788  0.02907021 -0.14109035
  0.01210359 -0.1850436   0.02004358  0.03893198  0.06305601 -0.02319679
 -0.04326034  0.00591453 -0.10462487 -0.05074805 -0.1504182   0.01525654
  0.06918819 -0.01018891  0.02317539 -0.09506267 -0.06039242  0.11180805
  0.0095405  -0.10466725 -0.0509086  -0.18092896 -0.02450538 -0.08663201
 -0.07712972 -0.00080951  0.10765804 -0.04551841 -0.08286542 -0.01110378
  0.08274718  0.06305473  0.01414887 -0.14069507 -0.05170505 -0.04127173
 -0.06244693  0.06728996  0.08358322  0.05085017 -0.09391297  0.07103299
  0.05802524  0.0247754  -0.04756982  0.00287238 -0.10527526  0.05204038
 -0.02731605  0.06806077 -0.1577914   0.13544388 -0.08401571  0.07078189
  0.12895425 -0.08100188  0.14813639  0.02573707 -0.06374004 -0.01801891
 -0.0839019   0.01346428 -0.04224009 -0.02892512 -0.

## FastText

In [23]:
sentences = remove_stop_words(train_sentence)
sentences.extend(remove_stop_words(valid_sentence))

model_ft = FastText(vector_size=100, window=5, min_count=1, sentences=sentences, epochs=10)
# model.save("word2vec.model")
# model = FastText.load("word2vec.model")

# access vector for one word
print(model_ft.wv['zombi'])

# get other similar words
# sims = model.wv.most_similar('zombi', topn=10)


#list the vocabulary words
words_ft = list(model_ft.wv.index_to_key)

print(len(words_ft))

[-0.05475635  0.0542057  -0.05570682  0.20506376  0.07276792 -0.07162417
  0.06998584  0.05062641  0.11029191 -0.17169708 -0.01079341 -0.05211193
 -0.14049539  0.2224241  -0.0438219   0.00732591 -0.08386838 -0.08949397
 -0.16162866 -0.10113554 -0.1742477   0.01439965 -0.08123017 -0.05248114
  0.03663433  0.00742886 -0.04388787 -0.00440948  0.12816195 -0.0104226
 -0.04942304  0.01094825  0.13993567 -0.0802836   0.01883134  0.05084011
 -0.02443593 -0.02598076 -0.11517685  0.05543267  0.15443198 -0.18191348
  0.10086079 -0.04423262 -0.06545889  0.01509274  0.00989179 -0.1522388
  0.00166808  0.15546855 -0.02202454 -0.05413138 -0.05568917  0.06896815
  0.0367025  -0.03525211  0.00708335 -0.07333439  0.0201804  -0.10147867
  0.20119257  0.07586314 -0.19908682  0.03630971  0.10305119  0.1624028
  0.01664571  0.02902112  0.10150111  0.13323516  0.08815565  0.02426819
  0.04080858 -0.1305007   0.15211287 -0.0443965   0.10403296 -0.05128287
 -0.09929251  0.06320065 -0.08365934 -0.11501861  0.03

# Test NN

In [ ]:

# Small LSTM Network to Generate Text for Alice in Wonderland

# load ascii text and covert to lowercase
filename = "wonderland.txt"
raw_text = open(filename, 'r', encoding='utf-8').read()
raw_text = raw_text.lower()
# create mapping of unique chars to integers
chars = sorted(list(set(raw_text)))
char_to_int = dict((c, i) for i, c in enumerate(chars))
# summarize the loaded data
n_chars = len(raw_text)
n_vocab = len(chars)
print("Total Characters: ", n_chars)
print("Total Vocab: ", n_vocab)
# prepare the dataset of input to output pairs encoded as integers
seq_length = 100
dataX = []
dataY = []
for i in range(0, n_chars - seq_length, 1):
 seq_in = raw_text[i:i + seq_length]
 seq_out = raw_text[i + seq_length]
 dataX.append([char_to_int[char] for char in seq_in])
 dataY.append(char_to_int[seq_out])
n_patterns = len(dataX)
print("Total Patterns: ", n_patterns)
# reshape X to be [samples, time steps, features]
X = np.reshape(dataX, (n_patterns, seq_length, 1))
# normalize
X = X / float(n_vocab)
# one hot encode the output variable
y = to_categorical(dataY)
# define the LSTM model
# You can now define your LSTM model. 
# Here, you define a single hidden LSTM layer with 256 memory units. 
# The network uses dropout with a probability of 20. 
# The output layer is a Dense layer using the softmax activation function to output a probability prediction for each of the 47 characters between 0 and 1.
model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2])))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')
# define the checkpoint
filepath="weights-improvement-{epoch:02d}-{loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]
# fit the model
# You can now fit your model to the data. Here, 
# you use a modest number of 20 epochs and a large batch size of 128 patterns.
model.fit(X, y, epochs=20, batch_size=128, callbacks=callbacks_list)

In [ ]:
# The network loss decreased almost every epoch, so the network could likely benefit from training for many more epochs.

# In the next section, you will look at using this model to generate new text sequences.

In [ ]:
# Load LSTM network and generate text
import sys
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import LSTM
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.utils import to_categorical
# load ascii text and covert to lowercase
filename = "wonderland.txt"
raw_text = open(filename, 'r', encoding='utf-8').read()
raw_text = raw_text.lower()
# create mapping of unique chars to integers, and a reverse mapping
chars = sorted(list(set(raw_text)))
char_to_int = dict((c, i) for i, c in enumerate(chars))
int_to_char = dict((i, c) for i, c in enumerate(chars))
# summarize the loaded data
n_chars = len(raw_text)
n_vocab = len(chars)
print("Total Characters: ", n_chars)
print("Total Vocab: ", n_vocab)
# prepare the dataset of input to output pairs encoded as integers
seq_length = 100
dataX = []
dataY = []
for i in range(0, n_chars - seq_length, 1):
 seq_in = raw_text[i:i + seq_length]
 seq_out = raw_text[i + seq_length]
 dataX.append([char_to_int[char] for char in seq_in])
 dataY.append(char_to_int[seq_out])
n_patterns = len(dataX)
print("Total Patterns: ", n_patterns)
# reshape X to be [samples, time steps, features]
X = np.reshape(dataX, (n_patterns, seq_length, 1))
# normalize
X = X / float(n_vocab)
# one hot encode the output variable
y = to_categorical(dataY)
# define the LSTM model
model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2])))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))
# load the network weights
filename = "weights-improvement-19-1.9435.hdf5"
model.load_weights(filename)
model.compile(loss='categorical_crossentropy', optimizer='adam')
# pick a random seed
start = np.random.randint(0, len(dataX)-1)
pattern = dataX[start]
print("Seed:")
print("\"", ''.join([int_to_char[value] for value in pattern]), "\"")
# generate characters
for i in range(1000):
 x = np.reshape(pattern, (1, len(pattern), 1))
 x = x / float(n_vocab)
 prediction = model.predict(x, verbose=0)
 index = np.argmax(prediction)
 result = int_to_char[index]
 seq_in = [int_to_char[value] for value in pattern]
 sys.stdout.write(result)
 pattern.append(index)
 pattern = pattern[1:len(pattern)]
print("\nDone.")


In [ ]:
# You will keep the number of memory units the same at 256 but add a second layer.

# ...
model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2]), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(256))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')